In [ ]:
import pandas as pd
df = pd.read_json('sommarioni_text_data_20250318_internal.json')
prov = df[df['owner'].apply(lambda x: 'provenienza' in x if not pd.isna(x) else False)]['owner']

# peculiar example:
# Suddetta di provenienza suddetta [CITTA' DI VENEZIA] di provenienza del Monastero di S. Giovanni di Torcello 
prov = prov.str.replace('Suddetta di provenienza suddetta ', '')
# peculiar exemple (missing the mention of the actual suddetta)
# Suddetta provenienza della sopressa Scuola della Carità
prov = prov.str.replace('Suddetta provenienza della sopressa Scuola della Carità', 'Suddetta [ERIZZO Niccolò q.m. Niccolò] provenienza della sopressa Scuola della Carità')
reg = r"(.*)provenienza (della|dalla|del|di|da|dei|delle|dello|dell'|dalle|\s)?(.*)"
curr_entity, old_entity = prov.str.extract(reg)[0], prov.str.extract(reg)[2]
people = pd.read_json('../../1808_Sommarioni/people_sommarioni_dataset_20240709.json')
all_parcels_ids = people.parcel_ids.values
flat_map = lambda x: [item for sublist in x for item in sublist]
people_vals = set(flat_map(all_parcels_ids))
df['is_people'] = df['unique_id'].apply(lambda x: x in people_vals)

In [ ]:
# cleaing
# step 1 removing "suddetta/suddetto" and the "[","]":
replace_vals = [
    'suddetta', 'Suddetta', 'suddetto', 'Suddetto', 'Suddette', 'suddette', 'Suddetti', 'suddetti', '[', ']'
]

def cleaning_suddett(s):
    if not pd.isna(s):
        for val in replace_vals:
            s = s.replace(val, '')
    return s

In [ ]:
curr_entity = curr_entity.apply(cleaning_suddett)
old_entity = old_entity.apply(cleaning_suddett)
df['curr_entity'] = curr_entity
df['old_entity'] = old_entity

In [ ]:
df.columns

In [ ]:
import os
import numpy as np 
import json

def carlo_dict_path_to_list(path: str) -> list[str]:
    with open(path) as f:
        res = json.load(f)
        return [list(v.items())[0][1] for v in res]
    

dicts_path = '1740_entities/ENT_dictionary/'
list_of_entities =  [(v, f.replace('.json', '')) for f in os.listdir(dicts_path) if f != 'entities.json' for v in carlo_dict_path_to_list(dicts_path+f)]
entity_class_dict = dict(list_of_entities)


In [ ]:
from Levenshtein import ratio

entities = carlo_dict_path_to_list(dicts_path+'entities.json')
soppress_fix = [
    'soppressa', 'sopressa','soppresso','sopresso', 
]

misc_fix = [
    ", e lughi sacri annessi",
    "e lughi sacri annessi",
    ", e luoghi sacri uniti"
    "e luoghi sagri annessi",
    ", e locali sagri annessi",
    "e locali sagri annessi",
    ', e luoghi sacri uniti',
    "e luoghi sacri uniti",
    "parrocchiale"
]

wrong_matches = {
    "Monastero di S. Andrea":"monastero di s. anna",
    "l MOnastero di S. Mauro": "monastero di santa marta",
    "suffraggio di Santa Ternita": "suffraggio di san geremia",
    "Convento di S. Secondo": "convento di san domenico",
    "Scuola di S. Stefano": "scola di san fantino",
    "Scola di San Gioacchino": "scola di san rocco",
    "Monastero di S. Andrea": "monastero di s. anna",
    "l Monastero di S. Salvatore": "monastero di s. alvise",
    "Chiesa di San Girolamo": "chiesa di san nicolo",
    "Monastero di S. Gerolamo di Venezia":"monastero di santa caterina di venezia",
    "Monastero di S. Daniele": "monastero di s. alvise",
    "suffraggio di Santa Ternita": "suffraggio di san geremia", 
    "Fraterna di S. Lorenzo":"fraterna di san vettore",
    "Convento di S. Secondo":"convento di san domenico",
    "Scuola di S. Stefano":"scola di san fantino",
    "CAPITOLO della Chiesa di S. Moisé":"capitolo della chiesa di san vidal",
    "Capitolo della Chiesa di S. Marciliano":"capitolo della chiesa di san nicolo",
    "Capitolo di S. Felice":"capitolo di san raffael",
    "Pievano di Santa Sofia":"pievano di santa marina",
    "Fraterna di S. Paulo":"fraterna di sant'angelo",
    "CONGREGAZIONE di S. Salvatore":"veneranda congregazione di san silvestro",
    "CAPITOLO di S. Salvadore":"capitolo di san lunardo",
    "CONGREGAZIONE di Carita'":"congregazione di santa maria",
    "CONGREGAZION di S. Salvatore":"veneranda congregazione di san silvestro",
    "SCOLA del Santissimo di S. Canziano":"scola del santissimo sacramento",
    "Monastero delle Monache di San Giuseppe":"venerando monastero delle monache di santa marta",
    "Fraterna dei Poveri Ebrei":"fraterna dei poveri vergognosi",
    "Fabrica della Chiesa di S. Maria Maddalena":"fabrica della chiesa di s. maria zobenigo",
    "Monastero delle Capuccine":"monastero del carmine",
    "Monastero delle Cappuccine":"monastero del carmine",
    "MORETTI Giovanni  Battista":"procuratia giovanni battista",
    "Chiesa Parrocchiale di S. Fantino":"chiesa di sant'antonino",
    "CAPITOLO della Chiesa Parrocchiale di S. Martino":"capitolo della chiesa di san nicolo",
    "FABBRICA della Chiesa Parrocchiale di S. Angelo":"fabrica della chiesa di san samuel",
    "CAPITOLO della Chiesa Parrocchiale di S. Bartolamio":"capitolo della chiesa di san nicolo",
    "CAPITOLO della Chiesa Parrocchiale di S. Moisé":"capitolo della chiesa di san vidal",
    "CAPITOLO della Chiesa Parrocchiale di S. Bartiolamio":"capitolo della chiesa di san nicolo",
    "CHIESA parrocchiale di S. Vito":"chiesa di san nicolo",
    "CAPITOLO della Chiesa Parrocchiale di S. Bartolomeo":"capitolo della chiesa di san nicolo",
    "CHIESA Parrocchiale di S. Maurizio":"chiesa di s. maria zobenigo",
    "Chiesa Parrocchiale di S. Luca, e lughi sacri annessi":"chiesa di san zuane",
    "CAPITOLO della Chiesa Parrocchiale di S. Pietro":"capitolo della chiesa di san nicolo",
    "CAPITOLO della Chiesa parrocchiale di S. Moisé":"capitolo della chiesa di san vidal",
    "CHIESA Parrocchiale di S. Benedetto di Venezia":"chiesa di san martino di venezia",
    "FABRICA della Chiesa Parrocchiale di S. Angelo di Venezia":"fabrica della chiesa di s. geremia",
    "Monastero di S. Martin": "monastero di santa marta"
}

def in_deptch_entity_matching(s:str, lev_thresh:float=0.8):
    matches = []
    if not pd.isna(s):
        original_s = s.strip()
        s = s.lower()
        if 'giovanni battista' in s and 'procuratia' not in s:
            # so many matches in owner between people called that and the "procuaratia giovanni battista"
            return None
        for sop in soppress_fix:
            if s.startswith(sop):
                s = s.replace(sop, '')
        for m in misc_fix:
            if m in s:
                s = s.replace(m, '')
        s = s.strip().replace('s.', 'san')
        for ent in entities:
            proc_ent = ent.replace('veneranda ', '').replace('venerando ', '').replace('s.', 'san').strip()
            r = ratio(proc_ent, s)
            if r > lev_thresh:
                matches.append([ent, r])
    # only selecting the highest possible match.
    best_match = None
    if len(matches) > 0:
        matches = sorted(matches, key=lambda x: x[1], reverse=True)
        best_match = matches[0][0]
        if original_s in wrong_matches.keys() and wrong_matches[original_s] == best_match:
            best_match = None
    return best_match

# to check:
# Monastero della Certosa di Venezia => padri della certosa di venezia (OK)
# Fraterna dei Poveri di S. Antonino => fraternita de vergognosi di san antonino (OK)
# Sesta di S. Rocco => chiesa di san rocco 
# Monastero di San Francesco di Paola => convento di san francesco di paola (OK)
# Monastero di S. Martin => monastero di santa marta, (FAUX!)
# CONGREGAZIONE di S. Paolo Appostolo => congregazione di san polo (OK)
# CONGREGAZIONE di S. Maria Zobenigo => congregazione di santa maria ()
# Capitolo e Fabbrica di S. Geremia => capitolo di san geremia (OK)

df['curr_entity_standardized'] = df['curr_entity'].apply(in_deptch_entity_matching)
df['old_entity_standardized'] = df['old_entity'].apply(in_deptch_entity_matching)
df['owner_entity_standardized'] = df['owner'].apply(in_deptch_entity_matching)
df['owner_standardized_entity_standardized'] = df['owner_standardised'].apply(in_deptch_entity_matching)

print('owner_standardized matches:', df['owner_standardized_entity_standardized'].notnull().sum())
print('curr_entity matches:', df['curr_entity_standardized'].notnull().sum())
print('old_entity matches:', df['old_entity_standardized'].notnull().sum())
print('owner matches:', df['owner_entity_standardized'].notnull().sum())

# NOTE:
# Commence par capitolo => institutions
# intervetir Sesta => Scuola 
# san polo sans indication => c'est l'apostolo
# COngregazione => probablement pas mal d'entités ad-hoc, à faire contrôler par isabella. 
# Congregazion di carita => nouvelle entité ad-hoc. 
# cas de la congréegazione di carita => contrôler avec la parcelle 1808 sur 1740 quel est le nom de l'institution. Pas besoin , cf. commentaire plus haut. 

# heuristic 

# Prompt: Institution detector. before any heurisitci

In [ ]:
df['old_entity_standardized_class'] = df['old_entity_standardized'].map(entity_class_dict)
df['owner_entity_standardized_class'] = df['owner_entity_standardized'].map(entity_class_dict)
df['owner_standardized_entity_standardized_class'] = df['owner_standardized_entity_standardized'].map(entity_class_dict)

# Congregazione standardisation

In [ ]:

cong_carita_standard_form = 'congregazione di carita'
def congregazione_detector(s) -> bool:
    if not pd.isna(s):
        s = s.lower()
        if 'congregazione' in s and 'carit' in s:
            return True
    return False

owner_congregazione = df[df['owner'].apply(congregazione_detector) & df['curr_entity'].isna()]
df.loc[owner_congregazione.index, 'owner_entity_standardized'] = cong_carita_standard_form

curr_entity_congregazione = df[df['curr_entity'].apply(congregazione_detector)]
df.loc[curr_entity_congregazione.index, 'curr_entity_standardized'] = cong_carita_standard_form

In [ ]:

# pd.concat([df[df['old_entity_standardized'].notnull()][['old_entity', 'old_entity_standardized']].drop_duplicates(), \
#            df[df['owner_entity_standardized'].notnull()][['owner', 'owner_entity_standardized']].drop_duplicates()
#            ]).to_csv('t.csv', index=False)

# df[df['parcel_number'].apply(lambda v: v.isalpha() if not pd.isna(v) else False) & (df['old_entity_standardized'].isnull() & df['owner_entity_standardized'].isnull() & df['owner'].notna())][['parcel_number', 'owner']].to_csv('t2.csv', index=False)

In [ ]:

isolate_people_from_alphabetical_parcel_number = False

if isolate_people_from_alphabetical_parcel_number:
    candidate_vals = df[df['parcel_number'].apply(lambda v: v.isalpha() if not pd.isna(v) else False) & (df['old_entity_standardized'].isnull() & df['owner_entity_standardized'].isnull() & df['owner'].notna())]
    candidate_vals[candidate_vals.is_people].owner.to_csv('people_candidates.csv', index=False)

people_but_institution = [
    "Casa Parrocchiale abitata dal Parroco della Chiesa di S. Moisé, porzione della quale viene affittata dal Parroco suddetto",
    "BENEFICIO del Diaconato nella Chiesa suddetta [di S. Moisé], attualmente goduto dal Sacerdote Giovanni VECCHINO",
    "DEMANIO, e per esso il Ministero della Finanza",
    "Benefizio suddetto [del Paroco di S. Marcuola attualmente Don Benedetto Schiavini]",
    "PISANI Pietro Vettor q. Vettor",
    "Benefizio del Secondo Prete di S. Fosca attulamente goduto dal Sacerdote Don Valentin Camauli",
    "MOCENIGO Alvise q. Pietro",
    "Oratorio sotto il titolo di San Giacomo Appostolo, detto San Giacomo di Rialto",
]

In [ ]:
from langchain_openai import ChatOpenAI
from ast import literal_eval
from langchain_core.prompts import (
    ChatPromptTemplate,
    FewShotChatMessagePromptTemplate,
)

import json

with open('../../../europeana/europeana/openapikey.txt', 'r') as f: 
    cont = f.readlines()

params = dict((v.replace('\n','').split('=')) for v in cont)
base_system_msg = '''
You are a helpful assistant that helps identify the type of entity from civic register written in italian.
You are given a list of examples of owners of land register that are either institutions or people.
Your task is to identify if the given sentence is an institution (true value).
Note that wenever you see the word "prebenda", "beneficio" or "provenienza" in the mention, it is an institution.
You should only answer with True or False.
'''

# Rajouter dans le prompt 

# prompt based identification of institution in all the cases that are not people:

few_shots_vals = [
("Casa Parrocchiale abitata dal Parroco della Chiesa di S. Moisé, porzione della quale viene affittata dal Parroco suddetto",True),
("PISANI Pietro Vettor q. Vettor",False),
("MOCENIGO Alvise q. Pietro",False),
("Oratorio sotto il titolo di San Giacomo Appostolo, detto San Giacomo di Rialto",True),
("Monastero di S. Giovanni Evangelista", True),
("ALBRIZZI Vincenzo e Fratelli", False),
('ANTELMI CIGUREO Francesca q.   , TRON Alvise q. Giovanni possessori indivisi', False),
('Scuola di S. Marco', True),
('l\'Ospitale di San Lodovico', True)
]


guesses_val = [{"register_entry":k, 'is_institution':v} for k,v in few_shots_vals]
guesses_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{register_entry}"),
        ("ai", "{is_institution}"),
    ]
)
guess_few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=guesses_prompt,
    examples=guesses_val,
)

guess_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", base_system_msg),
        guess_few_shot_prompt,
        ("human", "{register_entry}"),
    ]
)

chatgpt_llm = ChatOpenAI(
    openai_organization=params['oraganization_id'],
    openai_api_key=params['api_key'],
    model='gpt-4o-mini'
)
guess_agent = guess_prompt | chatgpt_llm

df_to_treat = df[df['owner'].notna() & df['old_entity'].isna() & df['curr_entity'].isna() & df['owner_entity_standardized'].isna()]
owner_to_identify = df_to_treat['owner'].to_dict()

from tqdm.notebook import tqdm
try:
    if os.path.exists('owner_to_identify.json'):
        with open('owner_to_identify.json', 'r') as f:
            owner_identified = json.load(f)
    else:
        owner_identified = {}
    print(len(owner_identified))
    for k, v in tqdm(owner_to_identify.items()):
        if str(k) not in owner_identified.keys():
            res = guess_agent.invoke({'register_entry': v})
            owner_identified[str(k)] = res.content
        if k % 1000 == 0:
            with open('owner_to_identify.json', 'w') as f:
                json.dump(owner_identified, f)
except Exception as e:
    print(e)

with open('owner_to_identify.json', 'w') as f:
    json.dump(owner_identified, f)

In [ ]:
owner_identified = {int(k): v for k, v in owner_identified.items()}
df['llm_guess'] = df.reset_index()['index'].apply(lambda v: owner_identified.get(v, None))
# df['llm_guess'] = df['llm_guess'].apply(lambda v: True if v == 'True' else False if v == 'False' else v)
 

In [ ]:


df[(df.llm_guess == 'True') & (~df.is_people)].owner.apply(cleaning_suddett).to_csv('institution_flagged_by_gpt.csv', index=False)

df[(df.llm_guess == 'False') & (~df.is_people)].owner.apply(cleaning_suddett)\
    .to_csv('not_people_not_institution.csv', index=False)

df[(df.llm_guess == 'True') & (df.is_people)].owner.apply(cleaning_suddett)\
    .to_csv('people_and_institution.csv', index=False)


In [ ]:
special_institution_spelling_dict = {
    "citta' di venezia": "città di venezia",
    "città di venezia": "città di venezia",
    "comune di venezia": "comune di venezia",
    "comune  di venezia": "comune di venezia",
    "ministero di finanze": "ministero delle finanze",
    "ministero di finanza": "ministero delle finanze",
    "ministero della guerra e marina": "ministero della guerra",
    "ministro della guerra": "ministero della guerra",
    "ministro della giustizia": "ministero della giustizia",
    "citta'": "città di venezia",

}

government_institution = {
    "ministero della guerra",
    "ministero delle finanze",
    "ufficio del censo",
    "ufficio del casatico di questa comune"
    "ministro della giustizia",
    "ministero dell'interno",
    "ministero della giustizia",
    "città di venezia",
    "comune di venezia",
    "demanio nazionale"
}

def government_institution_standardizer(s) -> bool:
    if not pd.isna(s):
        s = s.lower().strip()
        s = cleaning_suddett(s)
        for k,v in special_institution_spelling_dict.items():
            if k in s:
                s = v
        for val in government_institution:
            if val in s:
                return s
    return None

df_potential_government_institution = df[(df['llm_guess'] == 'True') & df['owner_entity_standardized'].isna() & df['curr_entity_standardized'].isna() & df['old_entity_standardized'].isna()]
government_institutions_standardized = df_potential_government_institution['owner'].apply(government_institution_standardizer)

In [ ]:
government_institutions_standardized
df.owner_entity_standardized.isna().sum()

In [ ]:
df.loc[government_institutions_standardized.index, 'owner_entity_standardized'] = government_institutions_standardized.values
df.owner_entity_standardized.isna().sum()

In [ ]:
# all the below are people mentions that the gpt tagged as institution.
people_correction = [
4137, #"ECHAR Riosa di Cadice rappresentata per procura di Grandis Francesco attualmente in deposito alla Direzione Generale del Demanio per le Divisioni del Detto 5 dicembre 1808",
21989, #"FRATERNA dei Poveri di S. Pantaleone, MONFERA' Elena q.m.     , CONTARINI Elisabeta q.m.   Monaca dello Spirito Santo, DADA q.m.    Conte, CASTAGNA Giovanni Battista q.m.", 
10324, #Albengo Bertucci
12659, #Antenier Avogadro di Treviso
18906, # CAPOVILLA Antonio q.m. Valerio BATTAGLIA Donato q.m. Sebastiano GUIZZETTI Tommaso Maria q.m. Giuseppe Direttori della Compagnia dei Mercanti della Camera del Purgo di Giusta Comune
22920, #Caripimi Zanchi
2642, # FORMENTI Sorelle di Ravenna
7989, #Fontana S.a Antonino,,,[]
18678, #R.R.P.P. Teatini volgarmente detti Tolentini,,,[]
18677, #"R.R.P.P. Teatini, volgarmente detti Tolentini",,,[

ZEN (ai Frari),,,[12966]
ZEN ....... q.m ...... alli Gesuiti,,,[15630]
"ZUCCATO Tiberio q.m. Angelo, CAPITOLO dei Sacerdoti della Chiesa Parochiale di S. Canciano",,,[22187]
]

# citta di venezia values:
# Sottoportico di Corte Colonne,,,[23171]
# Sottoportico di Corte di San Domenico,,,[23134]
# Sottoportico pubblico dei Santi,,,[23268]
# Sottoportico pubblico del Pistor,,,[23245]
# Sottoportico pubblico dell'Angelo,,,[22842]
# Sottoportico pubblico di Calle Cavallo,,,"[23206, 23209]"
# Sottoportico pubblico di Calle Schiavona,,,[23155]
# Sottoportico pubblico di Calle Vecchia,,,[23139]
# Sottoportico pubblico di Calle del Pistor,,,[23252]
# Sotto portico pubblico denominato di San Giovanni,,,[11679]


# normalize with "ministro della finanza"
# 313,"DEMANIO, e per esso il Ministero della Finanza",[23318],
# 314,"DEMANIO, e per esso il Ministero della Finanze","[17975, 17997]",

# normalize with "REALE CORONA"
# 487,REALE CORONA,"[23321, 23328, 23322]",
# 501,Reale Corona,"[3622, 3580, 3595, 3601, 3604, 3606, 3610, 3613, 3616, 3620, 3624, 3627, 3632, 3634, 3637, 3599]",
# 502,Reale Corrona,[3578],

In [ ]:
df[df.llm_guess.isnull()].iloc[0]

In [ ]:
dfcorr = pd.read_csv('standardizing_institution.csv')
dfcorr['normalized_version'] = ''
dfcorr[dfcorr.church_name.isna() & dfcorr.owner_supplementary.isna()]

In [ ]:
df['owner_cleaned'] = df['owner'].apply(cleaning_suddett).apply(lambda x: x.strip() if not pd.isna(x) else x)
investigate_df = df[(df.llm_guess == 'True') & df['owner_entity_standardized'].isna() & df['curr_entity_standardized'].isna() & df['old_entity_standardized'].isna()][['owner_cleaned', 'unique_id']].copy()
investigate_df = investigate_df.groupby('owner_cleaned').agg(list).reset_index()
investigate_df['church_name'] = ''
investigate_df['owner_supplementary'] = ''
investigate_df[['owner_cleaned', 'church_name', 'owner_supplementary', 'unique_id']].to_csv('standardizing_institution_wip.csv', index=False)